# forward difffusion process

![fig](../assets/DDPM.png)

In [2]:
"""
mean = sqrt(a_hat)*x_0
variance = sqrt(1-a_hat) * random_noise
"""

'\nmean = sqrt(a_hat)*x_0\nvariance = sqrt(1-a_hat) * random_noise\n'

In [5]:
import torch
import matplotlib.pyplot as plt

In [10]:
# define starting input
x0 = torch.randn(2,3,32,32)

In [21]:
# define variance scheduler betas
betas = torch.tensor([0.05,0.1,0.15,0.2,0.25])

In [22]:
# define timestep
t = torch.tensor([1,3])

In [23]:
# betas -> the amount of noise that are being applied at every timstep of diffusion process
# alphas -> 1-betas --> amount of original image's information that are being preserved affer a diffusion process step
alphas = 1 - betas


In [27]:
alphas_hat = torch.cumprod(alphas,axis=0)
alphas_hat

tensor([0.9500, 0.8550, 0.7268, 0.5814, 0.4360])

https://lilianweng.github.io/posts/2021-07-11-diffusion-models/